In [19]:
import pandas as pd
from sklearn.metrics import precision_score, recall_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV

In [12]:
# Wczytanie pliku CSV do DataFrame
data = pd.read_csv('SwissRe_AG_History.csv')

# Wyświetlenie pierwszych kilku wierszy danych
data.shape
data.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2023-01-03,87.639999,89.760002,87.260002,89.580002,78.203857,1304492
1,2023-01-04,89.699997,93.680000,89.699997,93.680000,81.783180,2001890
2,2023-01-05,93.500000,93.500000,92.059998,93.440002,81.573662,1277842
3,2023-01-06,93.559998,94.900002,93.300003,94.900002,82.848251,1336262
4,2023-01-09,94.300003,94.860001,93.820000,94.040001,82.097466,1016578


In [13]:
# Załóżmy, że kolumny 'Open', 'High', 'Low', 'Volume' będą cechami
X = data[['Open', 'High', 'Low', 'Volume']]
y = data['Close']

# Podział danych na zbiór treningowy i testowy
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [17]:
# Inicjalizacja modelu XGBoost (Random Forest)
xgb_model = XGBRegressor(objective='reg:squarederror', random_state=42)

# Definicja siatki hiperparametrów do przeszukania
param_grid = {
    'n_estimators': [100, 200, 300],  # liczba drzew
    'max_depth': [3, 4, 5],           # maksymalna głębokość drzewa
    'learning_rate': [0.01, 0.1, 0.2],# współczynnik uczenia
    'subsample': [0.8, 1],            # procent próbek do użycia dla pojedynczego drzewa
    'colsample_bytree': [0.8, 1]      # procent cech do użycia dla pojedynczego drzewa
}

# GridSearchCV do optymalizacji hiperparametrów
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, 
                           cv=3, scoring='neg_mean_squared_error', verbose=1)

# Trening modelu z przeszukaniem hiperparametrów
grid_search.fit(X_train, y_train)

print(grid_search)  # Wyświetla informacje o obiekcie grid_search

# Wyświetlenie najlepszych parametrów
print(grid_search.best_params_)  # Wyświetla najlepsze hiperparametry

Fitting 3 folds for each of 108 candidates, totalling 324 fits
GridSearchCV(cv=3,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, device=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    feature_types=None, gamma=None,
                                    grow_policy=None, importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=None, m...
                                    max_depth=None, max_leaves=None,
                                    min_child_weight=None, missing=nan,
                                    monotone_constraints=None,
                         

In [ ]:
# Predykcja na zbiorze testowym
y_pred = grid_search.best_estimator_.predict(X_test)

# Ocena modelu - obliczenie błędu średniokwadratowego
mse = mean_squared_error(y_test, y_pred)
print(f'Błąd średniokwadratowy: {mse}')